# Seizure Prediction LSTM Model

Simple LSTM model for predicting seizures 3 minutes in advance using HRV features.

Labels:
- 0: Normal periods
- 1: Pre-seizure (3 minutes before seizure onset)
- 2: During seizure

In [1]:
import numpy as np
import h5py
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix

2025-09-23 11:38:09.966398: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Load training data
with h5py.File('/Volumes/Seizury/HRV/sequences/train_sequences.h5', 'r') as f:
    X_train = f['X'][:]
    y_train = f['y'][:]

# Load validation data
with h5py.File('/Volumes/Seizury/HRV/sequences/val_sequences.h5', 'r') as f:
    X_val = f['X'][:]
    y_val = f['y'][:]

# Load test data
with h5py.File('/Volumes/Seizury/HRV/sequences/test_sequences.h5', 'r') as f:
    X_test = f['X'][:]
    y_test = f['y'][:]

print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
print(f"Label distribution - Train: {np.bincount(y_train)}")
print(f"Label distribution - Val: {np.bincount(y_val)}")
print(f"Label distribution - Test: {np.bincount(y_test)}")

Train: (1842884, 36, 22), Val: (297908, 36, 22), Test: (384912, 36, 22)
Label distribution - Train: [1832176    4716    5992]
Label distribution - Val: [296092    648   1168]
Label distribution - Test: [382708    852   1352]


### Option 1: 2 classes (normal vs pre-seizure)

In [3]:
# Convert to binary classification for seizure PREDICTION (3 minutes in advance)
# 0=Normal/During seizure, 1=Pre-seizure only (label 1)
# We want to predict BEFORE seizures happen, not during them
y_train_binary = (y_train == 1).astype(int)
y_val_binary = (y_val == 1).astype(int) 
y_test_binary = (y_test == 1).astype(int)

print(f"Prediction labels - Train: {np.bincount(y_train_binary)}")
print(f"Prediction labels - Val: {np.bincount(y_val_binary)}")
print(f"Prediction labels - Test: {np.bincount(y_test_binary)}")
print(f"Target: Predict label 1 (pre-seizure) vs everything else")

Prediction labels - Train: [1838168    4716]
Prediction labels - Val: [297260    648]
Prediction labels - Test: [384060    852]
Target: Predict label 1 (pre-seizure) vs everything else


### Option 2:  Include "During Seizure" as Positive Class

In [9]:
# Add this cell as an alternative to your current cell 4

# OPTION 2: Include seizure periods as positive class
# Hypothesis: Model needs to see seizure patterns to recognize pre-seizure patterns

y_train_combined = ((y_train == 1) | (y_train == 2)).astype(int)  # Both pre-seizure AND seizure
y_val_combined = ((y_val == 1) | (y_val == 2)).astype(int)
y_test_combined = ((y_test == 1) | (y_test == 2)).astype(int)

print(f"Combined approach - Labels 1+2 as positive:")
print(f"Train: {np.bincount(y_train_combined)}")
print(f"Val: {np.bincount(y_val_combined)}")
print(f"Test: {np.bincount(y_test_combined)}")

# This gives the model more positive examples to learn from
print(f"Positive examples increased from {np.sum(y_train == 1)} to {np.sum(y_train_combined)}")

Combined approach - Labels 1+2 as positive:
Train: [1832176   10708]
Val: [296092   1816]
Test: [382708   2204]
Positive examples increased from 4716 to 10708


### Option 3: Sequential Labeling (Progressive Distance to Seizure)

In [9]:
# Add this new cell after cell 4 in your notebook

# OPTION 1: Sequential Labeling - Distance to Seizure
# Create labels that show temporal progression toward seizure

def create_progressive_labels(y_original, timestamps=None):
    """
    Create progressive labels showing distance to seizure:
    0 = Normal (>5 min from seizure)
    1 = Far warning (3-5 min before seizure) 
    2 = Near warning (1-3 min before seizure)
    3 = Imminent (0-1 min before seizure)
    4 = During seizure
    """
    y_progressive = y_original.copy()
    
    # For now, use original labels but could be enhanced with timestamp analysis
    # 0 stays 0 (Normal)
    # 1 becomes 2 (Near warning - our current 3min target)  
    # 2 becomes 4 (During seizure)
    
    y_progressive[y_original == 1] = 2  # Pre-seizure becomes "near warning"
    y_progressive[y_original == 2] = 4  # Seizure becomes "during seizure"
    
    return y_progressive

# Create progressive labels
y_train_progressive = create_progressive_labels(y_train)
y_val_progressive = create_progressive_labels(y_val)
y_test_progressive = create_progressive_labels(y_test)

print("Progressive labeling distribution:")
print(f"Train: {np.bincount(y_train_progressive)}")
print(f"Val: {np.bincount(y_val_progressive)}")
print(f"Test: {np.bincount(y_test_progressive)}")

# Convert to binary: predict ANY warning (classes 1,2,3) vs normal/seizure (0,4)
y_train_warning = ((y_train_progressive >= 1) & (y_train_progressive <= 3)).astype(int)
y_val_warning = ((y_val_progressive >= 1) & (y_val_progressive <= 3)).astype(int)
y_test_warning = ((y_test_progressive >= 1) & (y_test_progressive <= 3)).astype(int)

print(f"\nWarning prediction labels:")
print(f"Train: {np.bincount(y_train_warning)}")
print(f"Val: {np.bincount(y_val_warning)}")
print(f"Test: {np.bincount(y_test_warning)}")

Progressive labeling distribution:
Train: [1832176       0    4716       0    5992]
Val: [296092      0    648      0   1168]
Test: [382708      0    852      0   1352]

Warning prediction labels:
Train: [1838168    4716]
Val: [297260    648]
Test: [384060    852]


In [4]:
# Verify our labeling strategy is correct for seizure prediction
print("\nOriginal 3-class distribution:")
print("Label 0 (Normal):", np.sum(y_train == 0), "sequences")
print("Label 1 (Pre-seizure - 3min before):", np.sum(y_train == 1), "sequences") 
print("Label 2 (During seizure):", np.sum(y_train == 2), "sequences")

print("\nOur prediction task:")
print("Predict: Label 1 (pre-seizure) = 1")
print("Everything else (normal + during seizure) = 0")
print("This gives 3-minute advance warning before seizures")


Original 3-class distribution:
Label 0 (Normal): 1832176 sequences
Label 1 (Pre-seizure - 3min before): 4716 sequences
Label 2 (During seizure): 5992 sequences

Our prediction task:
Predict: Label 1 (pre-seizure) = 1
Everything else (normal + during seizure) = 0
This gives 3-minute advance warning before seizures


In [ ]:
# Calculate class weights for imbalanced data
from sklearn.utils.class_weight import compute_class_weight

# For binary classification
classes = np.unique(y_train_binary)
class_weights_binary = compute_class_weight('balanced', classes=classes, y=y_train_binary)
class_weight_dict = {i: weight for i, weight in enumerate(class_weights_binary)}

print(f"Class weights for binary classification:")
print(f"Normal (0): {class_weight_dict[0]:.4f}")
print(f"Alert (1): {class_weight_dict[1]:.4f}")
print(f"Weight ratio: {class_weight_dict[1]/class_weight_dict[0]:.1f}:1")

Class weights for binary classification:
Normal (0): 0.3353
Alert (1): 130.2576
Weight ratio: 388.5:1


### Use a focal loss to handle class imbalance better

In [5]:
# Add this cell to replace your model compilation

import tensorflow as tf
from tensorflow.keras import backend as K

def focal_loss(gamma=2., alpha=0.25):
    """
    Focal Loss for addressing class imbalance.
    gamma: focusing parameter (higher = focus more on hard examples)
    alpha: weighting factor for rare class
    """
    def focal_loss_fixed(y_true, y_pred):
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
        p_t = tf.where(K.equal(y_true, 1), y_pred, 1 - y_pred)
        alpha_factor = K.ones_like(y_true) * alpha
        alpha_t = tf.where(K.equal(y_true, 1), alpha_factor, 1 - alpha_factor)
        cross_entropy = -K.log(p_t)
        weight = alpha_t * K.pow((1 - p_t), gamma)
        focal_loss_value = weight * cross_entropy
        return K.mean(focal_loss_value)
    return focal_loss_fixed

In [18]:
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Build improved LSTM model for seizure prediction
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.4),
    LSTM(64, return_sequences=True),
    Dropout(0.4),
    LSTM(32),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile without F1Score to avoid shape issues
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    #loss='binary_crossentropy',
    loss=focal_loss(gamma=2.0, alpha=0.99),  # High alpha for rare positive class
    metrics=['accuracy', 
             metrics.Precision(name="precision"),
             metrics.Recall(name="recall")]
)

model.summary()

/Users/pietrosaveri/Desktop/Pietro/•StartUps/Seizury/scr/use_features/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_6 (LSTM)                   │ (None, 36, 128)        │        77,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 36, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 36, 64)         │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 36, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 140,737 (549.75 KB)

 Trainable params: 140,737 (549.75 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
### MODEL FROM PAPER

from tensorflow.keras import metrics
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Build LSTM model following the specified architecture:
# 4 LSTM layers, 128 hidden nodes each, dropout 0.2 after each LSTM layer
model = Sequential([
    # First LSTM layer
    LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    
    # Second LSTM layer  
    LSTM(128, return_sequences=True),
    Dropout(0.2),
    
    # Third LSTM layer
    LSTM(128, return_sequences=True), 
    Dropout(0.2),
    
    # Fourth LSTM layer (final, no return_sequences)
    LSTM(128),
    Dropout(0.2),
    
    # Fully connected layer
    Dense(64, activation='relu'),
    
    # Output layer with sigmoid activation for binary classification
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    #loss='binary_crossentropy',
    loss=focal_loss(gamma=2.0, alpha=0.99),  # High alpha for rare positive class
    metrics=['accuracy', 
             metrics.Precision(name="precision"),
             metrics.Recall(name="recall")]
)

model.summary()

/Users/pietrosaveri/Desktop/Pietro/•StartUps/Seizury/scr/use_features/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 36, 128)        │        77,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 36, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 36, 128)        │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 36, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 36, 128)        │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 36, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 480,385 (1.83 MB)

 Trainable params: 480,385 (1.83 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
# Train with callbacks for better convergence
callbacks = [
    EarlyStopping(monitor='val_recall', patience=5, restore_best_weights=True, mode='max'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
]

history = model.fit(
    X_train, y_train_warning,
    epochs=2,
    batch_size=64,
    validation_data=(X_val, y_train_warning),
    #class_weight=class_weight_dict, #DO NOT USE IF FOCAL LOSS
    callbacks=callbacks,
    verbose=1
)

Epoch 1/2
  581/28796 ━━━━━━━━━━━━━━━━━━━━ 3:03:15 390ms/step - accuracy: 0.9772 - loss: 0.0018 - precision: 0.0019 - recall: 0.0159

KeyboardInterrupt: 

### Evalution (to edit with respect of y label option)

In [ ]:
# Evaluate on test set with optimized threshold
y_pred_prob = model.predict(X_test, verbose=0).flatten()

# Find optimal threshold based on F1 score
from sklearn.metrics import f1_score
thresholds = np.arange(0.1, 0.9, 0.05)
f1_scores = [f1_score(y_test_binary, (y_pred_prob > t).astype(int)) for t in thresholds]
optimal_threshold = thresholds[np.argmax(f1_scores)]

print(f"Optimal threshold: {optimal_threshold:.3f}")

# Make predictions with optimal threshold
y_pred = (y_pred_prob > optimal_threshold).astype(int)

# Evaluate
test_results = model.evaluate(X_test, y_test_binary, verbose=0)
print(f"\nTest Results:")
for i, metric in enumerate(model.metrics_names):
    print(f"{metric}: {test_results[i]:.4f}")

print(f"\nWith optimal threshold ({optimal_threshold:.3f}):")
print(classification_report(y_test_binary, y_pred, target_names=['Normal', 'Pre-seizure']))

print("\nConfusion Matrix:")
cm = confusion_matrix(y_test_binary, y_pred)
print(cm)
print(f"True Negatives: {cm[0,0]}, False Positives: {cm[0,1]}")
print(f"False Negatives: {cm[1,0]}, True Positives: {cm[1,1]}")

# Save the model
model.save('seizure_prediction_lstm.h5')
print(f"\nModel saved as seizure_prediction_lstm.h5")
print(f"Use threshold {optimal_threshold:.3f} for predictions")